# 4. Working with larger data sets

This workbook uses a large sample dataset in order to more realistically simulate the set up of the analysis process from start to finish. The dataset is not included directly with this code but can be downloaded from Zenodo - https://zenodo.org/record/3630511#.XjHhJGj7SUl

We assume that you have downloaded the data, unpacked it and placed it in the *example_data* folder before running this notebook.

The sample is a Titanium-64 Alloy (https://en.wikipedia.org/wiki/Ti-6Al-4V). The energy of the imaging beam was 89 KeV meaning a wavelenth of ~0.14 Å.

The experiment begins at room temperature, heats for approximately 2000 seconds, is held constant for about 1000 seconds, undergoes a high temperature deformation at around 3400 seconds from the start of the experiment and is then cooled. We expect primarily heaxagonal crystal structure ($\alpha$-phase) at room temperature, BCC crystal structure ($\beta$-phase) and high temperature and some small structural change as a result of the deformation.

Although we could follow the evolution of the cubic and hexagonal crystals in the material by fitting only a single peak for each, we want to consider as many of the possible orientations as possible to get better statistics.

We follow the beta structure by fitting peaks corresponding to the BCC Cubic Miller indices: 110, 200, 211, 220 and 310.

We follow the alpha structure by fitting hexagonal indices: 10-10, 0002, 10-11, 10-12, 11-20, 10-13, 11-22, 20-21 and 0004.


## 4.1. Calculating peak angles

Bragg's law gives the angle of scattering from a crystal lattice ($\theta$) as a function of the radiation wavelength planes ($\lambda$):

$$ \lambda = 2d \sin (\theta) $$

We have incoming radiation ($\theta$) of wavelength 0.14 Å. The Ti lattice constants are $a$ = 2.95 Å, $c$ = 4.68 Å.

### 4.1.1. Cubic crystal symmetry

For a cubic system the relation beween lattice spacing and the lattice constant ($a$) is:

$$ \frac{1}{d^2} = \frac{h^2 + k^2 + l^2}{a^2}  $$

Combining this relation with Bragg's law gives the relation between the scattering angle, radiation wavelength and lattice constant:

$$ \sin^2(\theta) = \frac{\lambda^2}{4a^2}(h^2 + k^2 + l^2) $$

For BCC the symmetry causes systematic absences in odd numbered hkl planes so we expect to see the 110, 200, 211, 220 and 310 peaks.

The approximate 2 theta scattering angles are then:

* 110: 3.85°
* 200: 5.44°
* 211: 6.66°
* 220: 7.70°
* 310: 8.60°

### 4.1.2. Hexagonal crystal symmetry

Using the Bravais-Miller system of notation for the hexagonal crystal symmetry (h, k, i, l) the relation between the lattice spacings and the lattice constants is:

$$ d = \frac{a}{\sqrt{\frac{4}{3}(h^2 + k^2 + hk) + \frac{a^2}{c^2}l^2)}} $$

Combining this with Bragg's law gives the relation between the scattering angle, radiation wavelength and lattice constants:

$$ \theta = \sin^{-1} \left( \frac{\lambda \sqrt{\frac{4}{3}(h^2 + k^2 + hk) + \frac{a^2}{c^2}l^2)}}{2a} \right) $$

Iterating through the combinations of indices gives us two theta angles of:

* 10-10: 3.14°
* 0002: 3.43°
* 10-11: 3.58°
* 10-12: 4.65°
* 11-20: 5.44°
* 10-13: 6.03°
* 20-20: 6.28°
* 11-22: 6.43°
* 20-21: 6.51°
* 0004: 6.86°


## 4.2 Finding the peaks

Lets take a look the diffraction pattern and see how it looks.

In [ ]:
%matplotlib inline

from xrdfit.spectrum_fitting import PeakParams, FitSpectrum, FittingExperiment

spectral_data = FitSpectrum('../example_data/example_data_large/adc_065_TI64_NDload_900C_15mms_00001.dat', 90)
spectral_data.plot_polar()

Now plot a single specturm

In [ ]:
spectral_data.plot(1)

Zoom in a bit.

In [ ]:
spectral_data.plot(1, x_range=(2.5, 10.5))

This looks reasonable. Since this is the first spectrum at the start of the experiment we expect high intensity hexagonal peaks and weak beta peaks. Using our calcualtions from above we can zoom in a section at a time and assign the peaks.

In [ ]:
spectral_data.plot(1, x_range=(3, 4), show_points=True)

These peaks fit with being 10-10, 0002, a weak 110 and 10-11. We should fit 0002, 110 and 10-11 as a triplet due to their proximity.

The 110 is a little way off our calculated value - about 8% lower than expected. This is due to alloying - inclusion of Vanadium in the beta phase. This is a constant percentage offset so we modify out original estimates of the peak positions to:

* 110: 3.55°
* 200: 5.01°
* 211: 6.13°
* 220: 7.08°
* 310: 8.92°

We can quickly plot the peak params to check they are OK, then do the fits and check the fit results.

In [ ]:
peak_params = [PeakParams('(10-10)', (3.02, 3.27)),
               PeakParams('(0002),(110),(10-11)',  (3.3, 3.75), [(3.4, 3.44), (3.52, 3.56), (3.57, 3.61)])]

spectral_data.plot_peak_params(peak_params, 1, show_points=True)

spectral_data.fit_peaks(peak_params, 1)
spectral_data.plot_fit('(10-10)')
spectral_data.plot_fit('(0002),(110),(10-11)')

That looks reasonable. Now on to the next section of the spectrum.

In [ ]:
spectral_data.plot(1, x_range=(4.4, 5.8), show_points=True)

These look good for 10-12, 200 and 11-20. Again lets quickly plot the fits to check they are OK.

In [ ]:
peak_params = [PeakParams('(10-12)', (4.54, 4.8)),
               PeakParams('(200)', (4.9, 5.10)),
               PeakParams('(11-20)', (5.35, 5.6))]

spectral_data.fit_peaks(peak_params, 1)
for fit in spectral_data.fitted_peaks:
    fit.plot()

Now moving up to the next section of the spectrum

In [ ]:
spectral_data.plot(1, x_range=(5.75, 7.5), show_points=True)

The first peak is the hexagonal 10-13. Going with our revised esimates for the cubic structures the small peak at 6.1° must be the 211. The next four strong peaks then correspond to 20-20, 11-22, 20-21 and 0004 in ascending order. There is a weak peak for the cubic 220 and then the next peak is likely hexagonal. I think we have got enough hexagonal peaks at this point but even though we dont want it, we still need to include it in the fit. The 220 peak is so close to this peak that we wont be able to relaibly fit it as a singlet. Lets just call our next alpha peak "?"

With the proximity of these peaks, they would be probably be good to fit as a doublet, singlet, doublet and singlet.

In [ ]:
peak_params = [PeakParams('(10-13),(211)', (5.9, 6.25), [(6.00, 6.05), (6.10, 6.15)]),
               PeakParams('(20-20)', (6.21, 6.4)),
               PeakParams('(11-22),(20-21)',  (6.37, 6.71), [(6.43, 6.47), (6.52, 6.56)]),
               PeakParams('(0004)',  (6.75, 6.95), [(6.82, 6.87)]),
               PeakParams('(220)(?)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)])]

spectral_data.plot_peak_params(peak_params, 1, show_points=True)

spectral_data.fit_peaks(peak_params, 1)
for fit in spectral_data.fitted_peaks:
    fit.plot()

Lets put all these peak params together and do a time fit.

## 4.3 Time fitting

### 4.3.1. Inital fit attempt

Lets start by fitting these peaks in every 100th frame to check nothing is odd before diving in and fitting the full data set.

In this case I choose to reuse the fits. Reusing the fits can cause problems if there is a significant difference between frames but can be faster if the fits are similar from frame to frame. It is best to try with and without and see which is quicker!

In this case reusing the fits is quicker becuase some of the triplet fits have close peaks with very different magnitudes so it is hard for the algorithm to guess good inital parameters to get the inflection point between them.

In [ ]:
frame_time = 1
file_string = '../example_data/example_data_large/adc_065_TI64_NDload_900C_15mms_{:05d}.dat'
first_cake_angle = 90
cakes_to_fit = 1
peak_params = [PeakParams('(10-10)', (3.02, 3.27)),
               PeakParams('(0002),(110),(10-11)',  (3.3, 3.75), [(3.4, 3.44), (3.52, 3.56), (3.57, 3.61)]),
               PeakParams('(10-12)', (4.54, 4.8)),
               PeakParams('(200)', (4.9, 5.10)),
               PeakParams('(11-20)', (5.35, 5.6)),
               PeakParams('(10-13),(211)', (5.9, 6.25), [(6.00, 6.05), (6.10, 6.15)]),
               PeakParams('(20-20)', (6.21, 6.4)),
               PeakParams('(11-22),(20-21)',  (6.37, 6.71), [(6.43, 6.47), (6.52, 6.56)]),
               PeakParams('(0004)',  (6.75, 6.95), [(6.82, 6.87)]),
               PeakParams('(220)(?)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)])]
max_frame = 5657
merge_cakes = False
frames_to_fit = range(1, max_frame, 100)
experiment = FittingExperiment(frame_time, file_string,first_cake_angle, cakes_to_fit, peak_params, merge_cakes, frames_to_fit)

experiment.run_analysis(reuse_fits=True)

You may have noticed that the progress bar does not fill linearly. For some timesteps the fitting stalls a little. This is because the fits are better at some points than others, sometimes many iterations of the fitting minimiser are required to get a good fit. This isn't necessarily a problem but excessive stalling can be indicitive of poor initial parameters.

After the fitting algorithm completes, if there are any peak fits which required more than 500 minimisation steps, they are highlighted in a report. These points should be checked in particular to see if there was a problem with the fitting. If you find that this report flags too many false positives you can increase the number of minimisation steps that triggers the report by providing the `evaluation_threshold` parameter to the `run_analysis` function.

In this case there are 5 peaks which the fitting has had trouble with at some point in the fitting process. The 20-20 and 0004 have only a couple of rough spots so this is probably nothing to worry about. However, the "(10-13),(211)", "(11-22),(20-21)" and "(220)(?)" peaks have multiple poor fits - these are probably worth checking.

### 4.3.2 Refining the "(10-13),(211)" peak

Lets do the "(10-13),(211)" peak first. The troublesome fits seem to be between timestep 12 and 31. First we can look at the fit parameters to see if there is anything unusual.

In [ ]:
experiment.plot_fit_parameter("(10-13),(211)", "maximum_1_center", show_points=True)
experiment.plot_fit_parameter("(10-13),(211)", "maximum_1_amplitude", show_points=True)
experiment.plot_fit_parameter("(10-13),(211)", "maximum_2_center", show_points=True)
experiment.plot_fit_parameter("(10-13),(211)", "maximum_2_amplitude", show_points=True)

If there is no error estimate availiable for a parameter, the data point on the graph is shown as a triangle. The lack of an error estimate may indicate a problem with the fit but not necessarily. The error estimate is made by inverting the curvature matrix of the fit. This will not work if changing the parameter does not significantly affect the fitting result or if changing the parameter results in an evalutation error.

In this case there is no error estimate for the center of the "(10-13)" peak but the values on the plot look reasonable, the amplitude of the "(10-13)" peak also looks OK and where the error can be calculated it is realtively small.

However the parameters for the "(222)" dont look good. The center parameter looks oddly stepped and the amplitude has massive error bars. Lets plot the actual fits now and see whats up. We know the trouble starts at about t=1000 (the 10th timestep) and stops at about t=4000 (the 40th frame) so we can limit the fits to these steps.

In [ ]:
experiment.plot_fits(peak_names=["(10-13),(211)"], timesteps=range(10, 40))

We can see that although the "(211)" peak is present at the begining of the experiment it disappears entirely on heating and doesnt reappear later in the exepriment. The doublet fit is still trying to fit a peak however and this is likely confusing the fit and why there was no error estimate for the "(10-13)" peak.

In this case it is probably better to ignore the 211 peak entirely as we cant get much information out of it. The "(10-13)" peak should be fine to fit on its own.

### 4.3.3 Refining the "(11-22),(20-21)" peak

The problems for this peak seem to be in the second half of the experiment. Lets plot the parameters first:

In [ ]:
experiment.plot_fit_parameter("(11-22),(20-21)", "maximum_1_center", show_points=True)
experiment.plot_fit_parameter("(11-22),(20-21)", "maximum_1_amplitude", show_points=True)
experiment.plot_fit_parameter("(11-22),(20-21)", "maximum_2_center", show_points=True)
experiment.plot_fit_parameter("(11-22),(20-21)", "maximum_2_amplitude", show_points=True)

There doesn't look to be anything wrong here other than the low amplitude of the peaks. It is still best to check the fits directly however since some problems are not readily apparent from just looking at the fit parameters.

In [ ]:
experiment.plot_fits(peak_names=["(11-22),(20-21)"], timesteps=range(20, 55))

The fits look fine here too, the only slight modification might be to move the left bound in slightly to avoid catching the edge of the 20-20 peak. The problem with this is that it would leave a pretty low number of baseline points so perhaps it would be best to fit the 20-20, 11-20 and 20-21 peaks as a triplet rather than as a singlet and doublet.

### 4.3.4 Refining the "(220)(?)" peak

The fitting really seems to be struggling with the "(220)(?)" peak.

In [ ]:
experiment.plot_fit_parameter("(220)(?)", "maximum_1_center", show_points=True)
experiment.plot_fit_parameter("(220)(?)", "maximum_1_amplitude", show_points=True)
experiment.plot_fit_parameter("(220)(?)", "maximum_2_center", show_points=True)
experiment.plot_fit_parameter("(220)(?)", "maximum_2_amplitude", show_points=True)

The fit of the "220" peak looks reasonable but it is the other peak that is breaking the fit.

In [ ]:
experiment.plot_fits(peak_names=["(220)(?)"], num_timesteps=30)

What seems to be happening here is the unidentifed peak disappears on heating and then reappears later. The problem is that after the fitting looses the peak it cannot find it again when it reappears. This is a result of reusing the fits. Because the parameters are poorly defined when the peak disappears, these odd parameters are carried over and the fitting just gets more and more confused each iteration. 

In this cases like this where the peaks vary so much over time it can be better to take a fresh guess at the parameters in each timestep, this means that if a fit fails in one timestep it will not directly affect the next timestep. As well as turning off the fit reuse globally we can turn it off for an individual peak by specifying "reuse_fits = False" in the `PeakParams`. This would would look like:

`PeakParams('(220)(?)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)]), reuse_fits=False]`

### 4.3.5. Implementing the refinements

Lets put all of these changes together and try the fits again.

We planned to remove the 211 peak, combine the 20-20, 11-20 and 20-21 into a triplet and set the 220 peak to not resue fits.

In [ ]:
peak_params = [PeakParams('(10-10)', (3.02, 3.27)),
               PeakParams('(0002),(110),(10-11)',  (3.3, 3.75), [(3.4, 3.44), (3.52, 3.56), (3.57, 3.61)]),
               PeakParams('(10-12)', (4.54, 4.8)),
               PeakParams('(200)', (4.9, 5.10)),
               PeakParams('(11-20)', (5.35, 5.6)),
               PeakParams('(10-13)', (5.85, 6.15)),
               PeakParams('(20-20)(11-22),(20-21)',  (6.2, 6.71), [(6.29, 6.34), (6.43, 6.47), (6.52, 6.56)]),
               PeakParams('(0004)',  (6.75, 6.95), [(6.82, 6.87)]),
               PeakParams('(220)(?)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)], reuse_fits=False)]

experiment = FittingExperiment(frame_time, file_string,first_cake_angle, cakes_to_fit, peak_params, merge_cakes, frames_to_fit)

experiment.run_analysis(reuse_fits=True)

That seems much better - notice that the analysis also runs quicker when poor fits are improved. Lets just check the "(220)(?)" Peak again to see that it has improved.

In [ ]:
experiment.plot_fit_parameter("(220)(?)", "maximum_1_center", show_points=True)
experiment.plot_fit_parameter("(220)(?)", "maximum_1_amplitude", show_points=True)
experiment.plot_fit_parameter("(220)(?)", "maximum_2_center", show_points=True, y_range=(7.05, 7.2))
experiment.plot_fit_parameter("(220)(?)", "maximum_2_amplitude", show_points=True, y_range=(0, 0.6))

The "220" maximum is now fit well and the other maximum is now fit well aside from when it is at a very low amplitude. That is probably the best we are going to get out of these peaks.

Also note the use of the `y-range` parameter for the `plot_fit_parameter` function. This can be useful when there are very large error bars that mess up the scale of a plot.

At this point we are ready to do the full fit and do some science.

## 4.4. Doing some science

The whole point of the fitting is to use the fitting parameters to do some science.

First lets do the fit again but at a finer resolution with more timesteps

In [ ]:
frame_time = 1
file_string = '../example_data/example_data_large/adc_065_TI64_NDload_900C_15mms_{:05d}.dat'
first_cake_angle = 90
cakes_to_fit = 1
peak_params = [PeakParams('(10-10)', (3.02, 3.27)),
               PeakParams('(0002),(110),(10-11)',  (3.3, 3.75), [(3.4, 3.44), (3.52, 3.56), (3.57, 3.61)]),
               PeakParams('(10-12)', (4.54, 4.8)),
               PeakParams('(200)', (4.9, 5.10)),
               PeakParams('(11-20)', (5.35, 5.6)),
               PeakParams('(10-13)', (5.85, 6.15)),
               PeakParams('(20-20)(11-22),(20-21)',  (6.2, 6.71), [(6.29, 6.34), (6.43, 6.47), (6.52, 6.56)]),
               PeakParams('(0004)',  (6.75, 6.95), [(6.82, 6.87)]),
               PeakParams('(220)(?)', (6.95, 7.35), [(7.05, 7.12), (7.16, 7.20)], reuse_fits=False)]
max_frame = 5657
merge_cakes = False
frames_to_fit = range(1, max_frame, 10)
experiment = FittingExperiment(frame_time, file_string,first_cake_angle, cakes_to_fit, peak_params, merge_cakes, frames_to_fit)

experiment.run_analysis(reuse_fits=True)

In [ ]:
experiment.plot_fit_parameter("(0004)", "maximum_1_center", show_points=True)
experiment.plot_fit_parameter("(0004)", "maximum_1_amplitude", show_points=True, y_range=(0, 1.3))

We can see that the angle of the 10-10 peak shifted down with increasing temperature (due to the increase in lattice spacing with thermal energy) and decreased in amplitude - almost dissapearing during the applied deformation section of the experiment.

In [ ]:
experiment.plot_fit_parameter('(0002),(110),(10-11)', "maximum_2_center", show_points=True) 
experiment.plot_fit_parameter('(0002),(110),(10-11)', "maximum_2_amplitude", show_points=True) 

The 110 peak, shows matching trends, the peak position shifts down a little but the amplitute greatly increases. Here we see the hexagonal ($\alpha$) structure being transformed to the cubic ($\beta$) structure.